In [2]:
import os,sys
import tensorflow as tf
from tensorflow import keras

In [3]:
pwd

'd:\\Chest-Cancer-classification\\expariment'

In [4]:
os.chdir('../')

In [5]:
pwd

'd:\\Chest-Cancer-classification'

In [6]:
from src.utils.main_utils import read_yaml,create_dir
from src.constant import *
from src.logging.logger import logging
from src.exception.exception import CustomException
import zipfile
import os,sys
import requests
import time
import urllib.request 
from zipfile import ZipFile

In [22]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainConfig:
    dir:Path
    model_path: Path
    update_base_model_path:Path
    train_data:Path
    epochs: int
    batch_size: int
    augmentation:bool
    image_size: list
    

In [31]:
class ModelTrainConfigManager:
    def __init__(self,
                 config_path=CONFIG_FILE_PATH,
                 pram_path=PARAMS_FILE_PATH):
        
        self.config=read_yaml(config_path)
        self.param=read_yaml(pram_path)

        create_dir([self.config.artifacts_root]) 

    def get_train_config(self)->ModelTrainConfig:
        train = self.config.model_train
        base_model = self.config.base_model
        param = self.param

        train_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chest-CT-Scan-data")
        
        create_dir([
            Path(train.dir)
        ])

        training_config = ModelTrainConfig(
            dir=Path(train.dir),
            model_path=Path(train.model_path),
            update_base_model_path=Path(base_model.update_base_model_path),
            train_data=Path(train_data),
            epochs=param.EPOCHS,
            batch_size=param.BATCH_SIZE,
            augmentation=param.AUGMENTATION,
            image_size=param.IMAGE_SIZE
        )

        return training_config

In [32]:
from tensorflow.keras.models import load_model

In [49]:
class Training:
    def __init__(self, config:ModelTrainConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.update_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.image_size[:-1],
            batch_size=self.config.batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.train_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.train_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
       
        optimizer = tf.keras.optimizers.SGD(learning_rate=0.003)
        loss=keras.losses.categorical_crossentropy

        self.model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

        self.model.fit(
        self.train_generator,
        epochs=self.config.epochs,
        steps_per_epoch=self.steps_per_epoch,
        validation_steps=self.validation_steps,
        validation_data=self.valid_generator
    )

        self.save_model(
            path=self.config.model_path,
            model=self.model
        )



In [50]:
try:
            config=ModelTrainConfigManager()
            train_config=config.get_train_config()
            train=Training(config=train_config)
            train.get_base_model()
            train.train_valid_generator()
            train.train()

except Exception as e:
            logging.info('errorr ',str(e))
            raise CustomException(sys,e)

Found 68 images belonging to 2 classes.
Found 275 images belonging to 2 classes.
Epoch 1/5


d:\Chest-Cancer-classification\cnn\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


27/27 ━━━━━━━━━━━━━━━━━━━━ 500s 18s/step - accuracy: 0.5060 - loss: 1.6315 - val_accuracy: 0.7333 - val_loss: 0.6713
Epoch 2/5
 1/27 ━━━━━━━━━━━━━━━━━━━━ 7:34 17s/step - accuracy: 0.8000 - loss: 0.6715

d:\Chest-Cancer-classification\cnn\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


27/27 ━━━━━━━━━━━━━━━━━━━━ 20s 105ms/step - accuracy: 0.8000 - loss: 0.3482 - val_accuracy: 0.6250 - val_loss: 0.3373
Epoch 3/5
27/27 ━━━━━━━━━━━━━━━━━━━━ 505s 19s/step - accuracy: 0.5890 - loss: 0.6655 - val_accuracy: 1.0000 - val_loss: 0.4481
Epoch 4/5
27/27 ━━━━━━━━━━━━━━━━━━━━ 20s 105ms/step - accuracy: 1.0000 - loss: 0.2539 - val_accuracy: 1.0000 - val_loss: 0.2249
Epoch 5/5
27/27 ━━━━━━━━━━━━━━━━━━━━ 491s 18s/step - accuracy: 0.6759 - loss: 0.6282 - val_accuracy: 0.6500 - val_loss: 0.5623
